In [2]:
import torch
from torch import nn
from torchvision import datasets, transforms
import torch.nn.functional as F

import os

In [3]:
!unzip -q /kaggle/input/generative-dog-images/all-dogs.zip -d /kaggle/working/all-dogs/

In [4]:
image_files = os.listdir('/kaggle/working/all-dogs')
print(f'Number of images in the dataset = {len(image_files)}')

Number of images in the dataset = 1


In [5]:
PATH = '/kaggle/working/all-dogs'
BATCH_SIZE = 32

In [6]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
train_data = datasets.ImageFolder(PATH, transform=transforms)
train_dataloader = torch.utils.data.DataLoader(train_data, 
                                               batch_size=BATCH_SIZE,
                                               shuffle=True)

In [12]:
class Generator(nn.Module):
    
    def __init__(self, n_z, n_feats, n_channels):
        super(Generator, self).__init__()
        
        self.conv1 = nn.ConvTranspose2d(in_channels=n_z, 
                                        out_channels=n_feats * 8, 
                                        kernel_size=4, 
                                        stride=1, 
                                        padding=0, 
                                        bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=n_feats * 8)
        
        self.conv2 = nn.ConvTranspose2d(in_channels=n_feats * 8,
                                        out_channels=n_feats * 8,
                                        kernel_size=4, 
                                        stride=2,
                                        padding=1,
                                        bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=n_feats * 8)
        
        self.conv3 = nn.ConvTranspose2d(in_channels=n_feats * 8, 
                                        out_channels=n_feats * 4,
                                        kernel_size=4,
                                        stride=2,
                                        padding=1,
                                        bias=False)
        self.bn3 = nn.BatchNorm2d(num_features=n_feats * 4)
        
        self.conv4 = nn.ConvTranspose2d(in_channels=n_feats * 4,
                                        out_channels=n_feats * 2,
                                        kernel_size=4,
                                        stride=2,
                                        padding=1,
                                        bias=False)
        self.bn4 = nn.BatchNorm2d(num_features=n_feats * 2)
        
        self.conv5 = nn.ConvTranspose2d(in_channels=n_feats * 2,
                                        out_channels=n_feats,
                                        kernel_size=4,
                                        stride=2,
                                        padding=1, 
                                        bias=False)
        self.bn5 = nn.BatchNorm2d(num_features=n_feats)
        
        self.conv6 = nn.ConvTranspose2d(in_channels=n_feats,
                                        out_channels=n_channels,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1,
                                        bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)))
        x = F.leaky_relu(self.bn2(self.conv2(x)))
        x = F.leaky_relu(self.bn3(self.conv3(x)))
        x = F.leaky_relu(self.bn4(self.conv4(x)))
        x = F.leaky_relu(self.bn5(self.conv5(x)))
        x = torch.tanh(self.conv6(x))
        
        return x

In [13]:
class Discriminator(nn.Module):
    
    def __init__(self, n_channels, n_feats):
        super(Discriminator, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=n_channels,
                               out_channels=n_feats,
                               kernel_size=4, 
                               stride=2,
                               padding=1,
                               bias=False)
        
        self.conv2 = nn.Conv2d(in_channels=n_feats,
                               out_channels=n_feats * 2,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=n_feats * 2)
        
        self.conv3 = nn.Conv2d(in_channels=n_feats * 2,
                               out_channels=n_feats * 4,
                               kernel_size=4,
                               stride=1,
                               padding=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(num_features=n_feats * 4)
        
        self.conv4 = nn.Conv2d(in_channels=n_feats * 4,
                               out_channels=n_feats * 8,
                               kernel_size=4,
                               stride=2,
                               padding=1,
                               bias=False)
        self.bn4 = nn.BatchNorm2d(num_features=n_feats * 8)
        
        self.conv5 = nn.Conv2d(in_channels=n_feats * 8,
                               out_channels=1,
                               kernel_size=4,
                               stride=1,
                               padding=0,
                               bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.02)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.02)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.02)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.02)
        x = torch.sigmoid(self.conv5(x))
        
        return x.view(-1, 1)

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

lr = 0.003
beta_1 = 0.5

netG = Generator(100, 32, 3).to(device)
netD = Discriminator(3, 48).to(device)

loss_fn = nn.BCELoss()

optimizerG = torch.optim.Adam(params=netG.parameters(),
                              lr=lr,
                              betas=(beta_1, 0.999))
optimizerD = torch.optim.Adam(params=netD.parameters(),
                              lr=lr,
                              betas=(beta_1, 0.999))

n_z = 100
fixed_noise = torch.randn(25, n_z, 1, 1, device=device)

real_label = 0.9 # label-smoothing
fake_label = 0.0
batch_size = train_dataloader.batch_size